In [31]:
from nemed.process import *
from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"

In [40]:
import nempy

In [61]:
cache = "E:/TEMPCACHE/"
start_time = "2022/01/01 00:00:00" # Start of historical period to collect data for
end_time = "2022/06/01 00:00:00" # End of historical period to collect data for
filter_units = None
filter_regions = None

In [101]:
region_stats = get_total_emissions_by_(start_time, end_time, cache, by="interval")
df = region_stats['Total_Emissions']
df = df[:-1]

Compiling data for table DISPATCHLOAD.
Returning DISPATCHLOAD.


In [111]:
#df = df.reset_index()
df2 = df.drop_duplicates(subset="Time")

In [112]:
temp = get_total_emissions_by_DI_DUID(
    start_time, end_time, cache, filter_units=None, filter_regions=None
)

Compiling data for table DISPATCHLOAD.
Returning DISPATCHLOAD.


In [123]:
filtime = "2022/02/01 18:35:00"
filtime = datetime.strptime(filtime,"%Y/%m/%d %H:%M:%S")

subs =temp[(temp['Time']==filtime) & (temp['Energy'] > 0)]
un = subs['DUID'].sort_values().to_list()
un

['ARWF1',
 'ARWF1',
 'ARWF1',
 'ARWF1',
 'ARWF1',
 'ARWF1',
 'ARWF1',
 'ARWF1',
 'BALBG1',
 'BALBG1',
 'BALBG1',
 'BALBG1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BALDHWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANGOWF1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BANN1',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARCALDN',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-1',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BARRON-2',
 'BERYLSF1',
 'BERYLSF1',
 'BERYLSF1',
 'BERYLSF1',
 'BERYLSF1',
 'BERYLSF1',
 'BERYLSF1',
 

In [102]:
df_nem = pd.DataFrame(index=df.index, columns=["NEM"],data=df.sum(axis=1))
df_nem

,NEM
Time,
2022-01-01 00:05:00,1355.51
2022-01-01 00:10:00,1359.27
2022-01-01 00:15:00,1348.89
2022-01-01 00:20:00,1332.11
2022-01-01 00:25:00,1330.27
...,...
2022-05-31 23:35:00,1380.00
2022-05-31 23:40:00,1378.08
2022-05-31 23:45:00,1374.36


### AEMO Summary Results Set

In [94]:
aemo = pd.read_csv('./CO2EII_SUMMARY_RESULTS.csv',header=1,usecols=[6,7,8,9,10])
aemo.head()

,SETTLEMENTDATE,REGIONID,TOTAL_SENT_OUT_ENERGY,TOTAL_EMISSIONS,CO2E_INTENSITY_INDEX
0,26/12/2021 0:00,NEM,428428.87030,270807.511500,0.6321
1,26/12/2021 0:00,NSW1,142048.65850,93962.043970,0.6615
2,26/12/2021 0:00,QLD1,133944.55410,102581.897900,0.7659
3,26/12/2021 0:00,SA1,28532.41922,1165.158262,0.0408
4,26/12/2021 0:00,TAS1,16089.71379,0.000000,0.0000


In [97]:
fil_start_dt = datetime.strptime(start_time,"%Y/%m/%d %H:%M:%S")
fil_end_dt = datetime.strptime(end_time,"%Y/%m/%d %H:%M:%S")

aemo['SETTLEMENTDATE'] = pd.to_datetime(aemo['SETTLEMENTDATE'])
sel_aemo = aemo[aemo['SETTLEMENTDATE'].between(fil_start_dt, fil_end_dt)]
sel_aemo_nsw = sel_aemo[sel_aemo['REGIONID']=='NSW1']
sel_aemo_nsw = sel_aemo_nsw.sort_values('SETTLEMENTDATE')
sel_aemo_nem = sel_aemo[sel_aemo['REGIONID']=='NEM']
sel_aemo_nem = sel_aemo_nem.sort_values('SETTLEMENTDATE')

In [103]:
fig = make_subplots(specs=[[{"secondary_y":True}]])
fig.update_layout(title='NEMED Calculated Total Emissions compared to AEMO reporting <br><sub>Daily Total Emissions - NSW</sub>',
				  template="simple_white",
                  xaxis_showgrid=True,
                  yaxis_showgrid=True,
                  xaxis_title="Time",
                  yaxis_title="Emissions (t CO2-e)")
#fig.update_yaxes(title_text="Price ($/MWh)", secondary_y=True)
#fig.update_yaxes(title_text="Emissions", secondary_y=False)
fig.add_trace(go.Scatter(x=df.index, y=df['NSW1'],name="Calculated total emissions",line={'color':
												'DarkSlateBlue'}),secondary_y=False)
fig.add_trace(go.Scatter(x=sel_aemo_nsw['SETTLEMENTDATE'], y=sel_aemo_nsw['TOTAL_EMISSIONS'],name='AEMO CDEII reporting',
						 line={'color':'DarkOrange'}),secondary_y=False)
fig.show()


In [100]:
fig = make_subplots(specs=[[{"secondary_y":True}]])
fig.update_layout(title='NEMED Calculated Total Emissions compared to AEMO reporting <br><sub>Daily Total Emissions - NEM</sub>',
				  template="simple_white",
                  xaxis_showgrid=True,
                  yaxis_showgrid=True,
                  xaxis_title="Time",
                  yaxis_title="Emissions (t CO2-e)")
#fig.update_yaxes(title_text="Price ($/MWh)", secondary_y=True)
#fig.update_yaxes(title_text="Emissions", secondary_y=False)
fig.add_trace(go.Scatter(x=df_nem.index, y=df_nem['NEM'],name="Calculated total emissions",line={'color':
												'DarkSlateBlue'}),secondary_y=False)
fig.add_trace(go.Scatter(x=sel_aemo_nem['SETTLEMENTDATE'], y=sel_aemo_nem['TOTAL_EMISSIONS'],name='AEMO CDEII reporting',
						 line={'color':'DarkOrange'}),secondary_y=False)
fig.show()
